In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#Reading data

akbank_data = pd.read_csv("C:\\Users\\ERALP\\OneDrive\\Desktop\\Software Engineering\\SWE599---Financial-Time-Series-Volatility-and-Return-Forecasting\\Code\\akbank_data.csv")


akbank_data = akbank_data[['Date', 'Hour', 'Open', 'High', 'Low', 'Close']]

#Converting Date and Hour to datetime object and setting it as index. This will help us to plot the data in a time series manner.

akbank_data['Datetime'] = pd.to_datetime(akbank_data['Date'] + " " + akbank_data['Hour'])

#Dropping Date and Hour columns
akbank_data.set_index('Datetime', inplace=True)
akbank_data.drop(['Date', 'Hour'], axis=1, inplace=True)

time_steps = 32

def create_sliding_windows(data, time_steps):
    X = []
    y = []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps, -1])
    return np.array(X), np.array(y)

#Normalizing the data
scaler = MinMaxScaler()
akbank_data_scaled = scaler.fit_transform(akbank_data[['Open', 'High', 'Low', 'Close']])

#Creating sliding windows
X, y = create_sliding_windows(akbank_data_scaled, time_steps)

#Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Converting the data into torch tensors
X_train = torch.tensor(X_train).float()
X_test = torch.tensor(X_test).float()
y_train = torch.tensor(y_train).float()
y_test = torch.tensor(y_test).float()

#Creating the Convolutional Neural Network model class

class StockPricePreditor(nn.Module):
    def __init__(self):
        super(StockPricePreditor, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=32, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64*21, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x=self.pool1(self.relu1(self.conv1(x)))
        x=self.pool2(self.relu2(self.conv2(x)))
        x=self.flatten(x)
        x=self.relu3(self.fc1(x))
        x=self.fc2(x)
        return x



#Creating the model object
model = StockPricePreditor()

#Defining the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Data loader

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

#Training the model

num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

#Validating the model

    valid_loss = 0.0
    with torch.no_grad():
        model.eval()
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            valid_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {valid_loss/len(test_loader)}")

#Evaluating the model
model.eval()
test_loss = 0.0
predictions = []
true_values = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        test_loss += loss.item()
        predictions.extend(outputs.squeeze().numpy())
        true_values.extend(labels.numpy())

print(f"Test Loss: {test_loss/len(test_loader)}")

#Converting the predictions and true values to the original scale

predictions = np.array(predictions).reshape(-1, 1)
true_values = np.array(true_values).reshape(-1, 1)
predictions = scaler.inverse_transform(np.hstack([np.zeros((predictions.shape[0], 3)), predictions]))
true_values = scaler.inverse_transform(np.hstack([np.zeros((true_values.shape[0], 3)), true_values]))

# Calculating the mean squared error and mean absolute error
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(true_values[:,-1], predictions[:,-1])
mae = mean_absolute_error(true_values[:,-1], predictions[:,-1])

print(f"MSE: {mse}, MAE: {mae}")

#Plotting the results
plt.figure(figsize=(15, 5))
plt.plot(true_values, label='True Values')
plt.plot(predictions, label='Predictions')
plt.title('True Values vs Predictions')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.show()

RuntimeError: Given groups=1, weight of size [32, 4, 5], expected input[32, 32, 4] to have 4 channels, but got 32 channels instead